In [1]:
COLORS = {
    'reset': '\033[0m',

    'red': '\033[31m',
    'green': '\033[32m',
    'yellow': '\033[33m',
}

def GetColor(color_name: str) -> str:
    return COLORS.get(color_name, COLORS['reset'])

def Colored(text: str, color_name: str = 'green') -> str:
    return f"{GetColor(color_name)}{text}{GetColor('reset')}"


In [6]:
import numpy as np

def TransitionMatrix(matrix: list) -> np.ndarray:
    return np.array(object=matrix)

def PeopleVector(people_industry: int, people_academia: int) -> np.ndarray:
    return np.array([people_industry, people_academia])

def CalculatePeople(percent: float, total_people: int) -> int:
    return percent * total_people, (1 - percent) * total_people

def PeopleInitialState(
    total_people: int, percentage_industry: float = None,  percentage_academia: float = None, 
) -> np.ndarray:

    if percentage_industry is not None:
        people_industry, people_academia = CalculatePeople(percentage_industry, total_people)
        return PeopleVector(people_industry, people_academia)

    elif percentage_academia is not None:
        people_academia, people_industry = CalculatePeople(percentage_academia, total_people)
        return PeopleVector(people_industry, people_academia)
    
    else:
        raise ValueError("Either percentage_industry or percentage_academia must be provided.")

def CalculatePeopleNextYear(transition_matrix: np.ndarray, people: np.ndarray) -> np.ndarray:
    return np.dot(transition_matrix, people)

def SimulateYears(
    initial_year: int, period_of_time: int,
    people: np.ndarray, transition_matrix: np.ndarray, step: int = 1,
) -> list[tuple[int, np.ndarray]]:
    
    history = [(initial_year, people.copy())]

    start = initial_year + 1
    stop  = (initial_year + period_of_time) + 1
    step = step if step > 0 else 1

    for year in range(start, stop, step):
        people = CalculatePeopleNextYear(transition_matrix, people)
        history.append((year, people.copy()))

    return history

def UnpackData(history: list[tuple[int, np.ndarray]]) -> tuple[list[int], list[float], list[float]]:
    X_axis = [year for year, _ in history]
    Y_axis_industry = [people[0] for _, people in history]
    Y_axis_academia = [people[1] for _, people in history]
    return X_axis, Y_axis_industry, Y_axis_academia

def GetTicks(initial_value: float, final_value: float, num_ticks: int = 21, dtype: object = int) -> np.ndarray:
    return np.linspace(initial_value, final_value, num_ticks, dtype=dtype)

def GetMinMaxValues(*args: list[float], delta_up: float = 1.1, delta_down: float = 0.8) -> tuple[float, float]:
    return min(min(arg) for arg in args) * delta_down, max(max(arg) for arg in args) * delta_up

def GetGraphicParameters(
    people_history: list[tuple[int, np.ndarray]],
    initial_year: int, period_of_time: int,
) -> dict[str, object]:
    
    X_axis, Y_axis_industry, Y_axis_academia = UnpackData(people_history)
    Y_min, Y_max = GetMinMaxValues(Y_axis_industry, Y_axis_academia)
    
    return X_axis, Y_axis_industry, Y_axis_academia, {
        'Y_min': Y_min, 'Y_max': Y_max,
        'X_ticks': GetTicks(initial_value = initial_year, final_value = initial_year + period_of_time), 
        'Y_ticks': GetTicks(initial_value=Y_min, final_value=Y_max, dtype=float), 
    }

def FindChangeYears(
    X_axis: list[int],
    Y_axis_industry: list[float],
    Y_axis_academia: list[float]
) -> list[int]:

    change_years = []
    current_leader = None

    for year, industry, academia in zip(X_axis, Y_axis_industry, Y_axis_academia):
        leader = None
        if industry > academia:
            leader = 'industry'
        elif academia > industry:
            leader = 'academia'
        else:
            leader = 'equal'

        if current_leader is not None and leader != current_leader:
            change_years.append(year)

        current_leader = leader
        
    return change_years


## `Absolute Values`: 

In [18]:
T = [
    [0.99, 0.2], 
    [0.01, 0.8],
]

transitionMatrix = TransitionMatrix(matrix=T)


## `Initial Values`:


In [19]:
totalPeople = 1_000
percentagePeopleIndustry = 0.1

initialYear = 2025
periodOfTime = 100
finalYear = initialYear + periodOfTime


In [23]:
def Simulation(
    total_people: int,
    percentage_people_industry: float,
    initial_year: int,
    period_of_time: int,
    transition_matrix: np.ndarray,
):
    
    people = PeopleInitialState(
        total_people=total_people, 
        percentage_industry=percentage_people_industry,
    )

    peopleHistory = SimulateYears(
        people=people,
        initial_year=initial_year,
        period_of_time=period_of_time,
        transition_matrix=transition_matrix,
    )

    X_axis, Y_axis_industry, Y_axis_academia, Parameters = GetGraphicParameters(
        people_history=peopleHistory,
        initial_year=initial_year,
        period_of_time=period_of_time,
    )

    change_years = FindChangeYears(X_axis, Y_axis_industry, Y_axis_academia)
    difference_change_years = [abs(initial_year - year) for year in change_years]

    print(f"Years when the leader changed: {Colored(f'{change_years}', 'yellow')}")
    print(f"Difference from initial year: {Colored(f'{difference_change_years}', 'yellow')}")


    return X_axis, Y_axis_industry, Y_axis_academia, Parameters


def IncreasePercentageIndustry(
    initial_percentage: float, increase_step: float, max_percentage: float = 1.0
) -> list[float]:

    percentages = []
    current_percentage = initial_percentage

    while current_percentage <= max_percentage:
        percentages.append(round(current_percentage, 4))
        current_percentage += increase_step

    return percentages

In [24]:

Responses = Simulation(
    total_people=totalPeople,
    initial_year=initialYear,
    period_of_time=periodOfTime,
    transition_matrix=transitionMatrix,
    percentage_people_industry=percentagePeopleIndustry,
)
Responses

Years when the leader changed: [2028]
Difference from initial year: [3]


([2025,
  2026,
  2027,
  2028,
  2029,
  2030,
  2031,
  2032,
  2033,
  2034,
  2035,
  2036,
  2037,
  2038,
  2039,
  2040,
  2041,
  2042,
  2043,
  2044,
  2045,
  2046,
  2047,
  2048,
  2049,
  2050,
  2051,
  2052,
  2053,
  2054,
  2055,
  2056,
  2057,
  2058,
  2059,
  2060,
  2061,
  2062,
  2063,
  2064,
  2065,
  2066,
  2067,
  2068,
  2069,
  2070,
  2071,
  2072,
  2073,
  2074,
  2075,
  2076,
  2077,
  2078,
  2079,
  2080,
  2081,
  2082,
  2083,
  2084,
  2085,
  2086,
  2087,
  2088,
  2089,
  2090,
  2091,
  2092,
  2093,
  2094,
  2095,
  2096,
  2097,
  2098,
  2099,
  2100,
  2101,
  2102,
  2103,
  2104,
  2105,
  2106,
  2107,
  2108,
  2109,
  2110,
  2111,
  2112,
  2113,
  2114,
  2115,
  2116,
  2117,
  2118,
  2119,
  2120,
  2121,
  2122,
  2123,
  2124,
  2125],
 [np.float64(100.0),
  np.float64(279.0),
  np.float64(420.40999999999997),
  np.float64(532.1239),
  np.float64(620.3778810000001),
  np.float64(690.0985259900001),
  np.float64(745.17783553